In [3]:
from bokeh.models import ColumnDataSource, FactorRange, Whisker, HoverTool
from bokeh.palettes import colorblind  # Import colorblind palette
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, output_file

output_notebook()  # Outputs plots inline in Jupyter Notebook

# Data preparation
avg_values = {
    '10k': [16.5, 18.7333, 18.033, 19.7666, 18.8333, 18.9666],
    '130k': [24.5333, 27.5333, 32.76666, 31.46666, 30.43333, 38.96666]
}
std_values = {
    '10k': [5.823, 5.39711, 4.87499, 3.3534, 4.0831, 3.9368],
    '130k': [16.841, 16.189, 15.9576, 18.1433, 13.8363, 16.9597]
}

datasets = ['10k', '130k']
nudged = ['0', '1', '2', '3', '4', '5']

x = [(dataset, nudge) for dataset in datasets for nudge in nudged]
counts_10k = avg_values['10k']
counts_130k = avg_values['130k']
lower_bounds_10k = [c - s for c, s in zip(counts_10k, std_values['10k'])]
upper_bounds_10k = [c + s for c, s in zip(counts_10k, std_values['10k'])]
lower_bounds_130k = [c - s for c, s in zip(counts_130k, std_values['130k'])]
upper_bounds_130k = [c + s for c, s in zip(counts_130k, std_values['130k'])]

# Correct concatenation
counts = counts_10k + counts_130k
lower_bounds = lower_bounds_10k + lower_bounds_130k
upper_bounds = upper_bounds_10k + upper_bounds_130k

source = ColumnDataSource(data=dict(
    x=x,
    counts=counts,
    lower=lower_bounds,
    upper=upper_bounds
))

# Define the maximum upper bound for adjusting the plot's y-range dynamically
max_upper_bound = max(upper_bounds)

palette = colorblind['Colorblind'][5]  # Using 5 colors from the colorblind palette

p = figure(x_range=FactorRange(*x), height=350,
           toolbar_location='right', output_backend="svg")

# Add hover tool
hover = HoverTool()
hover.tooltips = [
    ("Index", "$index"),
    ("Count", "@counts"),
    ("Lower Bound", "@lower"),
    ("Upper Bound", "@upper")
]
p.add_tools(hover)

p.y_range.start = 0
p.y_range.end = max_upper_bound + 5  # Adding padding to ensure all error bars are visible

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=datasets, start=0, end=1))

p.add_layout(Whisker(source=source, base='x', upper='upper', lower='lower', line_color='black', level='overlay'))
p.yaxis.axis_label = "Number of top-1% found"
p.xaxis.axis_label = "Number of enriched molecules"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "gray"
p.ygrid.grid_line_dash = [6, 4]
p.ygrid.grid_line_alpha = 0.6

show(p)
output_file('nudging.html')

Loading BokehJS ...